# Map template retinotopy to individual

Based on work by the [Geoffrey Aguirre Lab](https://cfn.upenn.edu/aguirre/wiki/public:retinotopy_template), and Benson et al.

In [4]:
SUBJECTS_DIR=/Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir
subject=s18
do_surfreg=false  # takes a long time
clobber_surfreg=false  # redo

## Register subject to symmetric `fsaverage`

This takes quite a bit of time:

* 17 min for `--lh` (Macbook Pro 2013, 16 GB RAM)
* 20 min for `--lh xhemi` (Macbook Pro 2013, 16 GB RAM)

`surfreg` registers each hemishpere of the `$subject` to the symmetric atlas. See the [Freesurfer page](http://surfer.nmr.mgh.harvard.edu/fswiki/Xhemi) for details on how (why) the right hemisphere is dealt with in the particular way it is. See also the [Benson et al. procedure](https://cfn.upenn.edu/aguirre/wiki/public:retinotopy_template).

__NB: The `--lh` below are checked and correct!__

In [5]:
if [ $do_surfreg = true ]; then
    if [ $clobber_surfreg = true ]; then
        rm ${SUBJECTS_DIR}/${subject}/surf/lh.fsaverage_sym.sphere.reg
        rm ${SUBJECTS_DIR}/${subject}/xhemi/surf/lh.fsaverage_sym.sphere.reg
    fi
    surfreg --s $subject --t fsaverage_sym --lh
    surfreg --s $subject --t fsaverage_sym --lh --xhemi
fi

Fri Dec  9 15:21:16 CET 2016

setenv SUBJECTS_DIR /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir
cd /Users/cjb/projects/CFF_Retinotopy/scripts
/Users/cjb/UnixApps/freesurfer/bin/surfreg --s s18 --t fsaverage_sym --lh

Darwin hathor 15.5.0 Darwin Kernel Version 15.5.0: Tue Apr 19 18:36:36 PDT 2016; root:xnu-3248.50.21~8/RELEASE_X86_64 x86_64
#@# lh Fri Dec  9 15:21:16 CET 2016
#CMD# mris_register -curv -annot aparc.annot /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir/s18/surf/lh.sphere /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir/fsaverage_sym/lh.reg.template.tif /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir/s18/surf/lh.fsaverage_sym.sphere.reg
using smoothwm curvature for final alignment
zeroing medial wall in aparc.annot
$Id: mris_register.c,v 1.59 2011/03/02 00:04:33 nicks Exp $
  $Id: mrisurf.c,v 1.693.2.7 2013/05/12 22:28:01 nicks Exp $
reading surface from /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir/s18/surf/lh.sphere...
reading colortable from annotation

## Coregister angle-, eccentricity- and area-templates to individual

Requires these files:
* `angle-template-2.5.sym.mgh`
* `eccen-template-2.5.sym.mgh`
* `areas-template-2.5.sym.mgh`

In [6]:
sym_files=(angle-template-2.5.sym.mgh \
           eccen-template-2.5.sym.mgh \
           areas-template-2.5.sym.mgh)

In [7]:
# for each of the symmetric templates
for sf in ${sym_files[@]}
do
    prefix="$( cut -d '-' -f 1 <<< "$sf" )"  # get template short name
    # separately for each hemisphere
    for hemi in 'lh' 'rh'
    do
        outname=${SUBJECTS_DIR}/${subject}/label/${hemi}.benson.${prefix}.mgh

        # "target subject" for right hemisphere is xhemi
        trgsubj=$subject
        if [[ $hemi == rh ]]; then
            trgsubj=${trgsubj}/xhemi
        fi
        
        # NB! --hemi is always lh, rh is xhemi!
        cmd="mri_surf2surf --srcsubject fsaverage_sym --srcsurfreg sphere.reg \
             --trgsubject ${trgsubj} --trgsurfreg fsaverage_sym.sphere.reg \
             --hemi lh --sval $sf --tval $outname"
        echo $cmd
    #     eval "$cmd"
    done
done

mri_surf2surf --srcsubject fsaverage_sym --srcsurfreg sphere.reg --trgsubject s18 --trgsurfreg fsaverage_sym.sphere.reg --hemi lh --sval angle-template-2.5.sym.mgh --tval /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir/s18/label/lh.benson.angle.mgh
mri_surf2surf --srcsubject fsaverage_sym --srcsurfreg sphere.reg --trgsubject s18/xhemi --trgsurfreg fsaverage_sym.sphere.reg --hemi lh --sval angle-template-2.5.sym.mgh --tval /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir/s18/label/rh.benson.angle.mgh
mri_surf2surf --srcsubject fsaverage_sym --srcsurfreg sphere.reg --trgsubject s18 --trgsurfreg fsaverage_sym.sphere.reg --hemi lh --sval eccen-template-2.5.sym.mgh --tval /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir/s18/label/lh.benson.eccen.mgh
mri_surf2surf --srcsubject fsaverage_sym --srcsurfreg sphere.reg --trgsubject s18/xhemi --trgsurfreg fsaverage_sym.sphere.reg --hemi lh --sval eccen-template-2.5.sym.mgh --tval /Users/cjb/projects/CFF_Retinotopy/fs_subjects_dir/s18/label/